In [2]:
# %load ../start.py
# Load useful extensions

# Activate the autoreload extension for easy reloading of external packages
%reload_ext autoreload
%autoreload 2

# Trun on the water mark
%reload_ext watermark
%watermark -u -d -g

# Load ipycache extension
%reload_ext ipycache
from ipycache import CacheMagics
CacheMagics.cachedir = '../cachedir'

# Add project library to path
import sys
sys.path.insert(0, '../../lib/python')

# The usual suspects
import os
import numpy as np
import pandas as pd

# plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set_context('poster')

# Turn off scientific notation
np.set_printoptions(precision=5, suppress=True)


last updated: 2017-09-25 
Git hash: 62c1f50f252a1e772d642848d759ee07e8a88396


In [3]:
# %load ../../bin/load.py
from pymongo import MongoClient
with open('/home/fearjm/Projects/ncbi_remap/output/.mongodb_host', 'r') as fh:
    host = fh.read().strip()
client = MongoClient(host=host, port=27022)
db = client['sra2']
remap = db['remap']

In [4]:
remap.find({'runs.aln_flags': 'complete'}).count()

9425

In [36]:
srxs = pd.read_csv('../../output/modENCODE_sampletable.tsv', sep='\t').srx.unique().tolist()
srrs = pd.read_csv('../../output/modENCODE_sampletable.tsv', sep='\t').srr.unique().tolist()
len(srxs)

1707

In [35]:
list(remap.aggregate([
    {'$unwind': '$runs'},
    {
        '$match': {
           'runs.pre_aln_flags': {'$nin': ['download_bad', 'alignment_bad', 'complete', 'abi_solid']},
           '_id': {'$in': srxs},
        }
    },
]))

[{'_id': 'SRX148403',
  'bioproject': None,
  'contacts': [],
  'papers': [],
  'runs': {'avgReadLen': {},
   'libsize': {},
   'md5': {},
   'pre_aln_flags': [],
   'srr': 'SRR497720'},
  'sample': {'biosample': 'SAMN00996465', 'srs': 'SRS334143'},
  'sra': 'SRA052953',
  'srp': 'SRP013225'}]

In [42]:
list(remap.aggregate([
    {'$unwind': '$runs'},
    {
        '$match': {
            'runs.srr': {'$in': srrs},
            'runs.pre_aln_flags': []
        }
    },
    {'$count': 'cnt'}
]))

[{'cnt': 1}]